# Setup


In [2]:
MODEL_UID = "unsloth/llama-3.2-1B-bnb-4bit"
DATASET_UID = "allenai/tulu-3-sft-personas-algebra"
MAX_STEPS = 1  # edit this after debugging

print("Done")

Done


In [4]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install -e ./lm-evaluation-harness/.
!pip install unsloth transformers datasets

fatal: destination path 'lm-evaluation-harness' already exists and is not an empty directory.
Obtaining file:///Users/paul/Documents/Uni/M.Sc.%20CogSys/4.%20Semester/PM/pm-25/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lm_eval (pyproject.toml) ... done
  Created wheel for lm_eval: filename=lm_eval-0.4.8-0.editable-py3-none-any.whl size=24841 sha256=7729cac4a3ad5427bbb9db8f0f5d6b6b7511c245a164b770e80151257291d611
  Stored in directory: /private/var/folders/wg/kwksxbdd0wd0x0hjzrpjw1tc0000gq/T/pip-ephem-wheel-cache-a3lo_9nh/wheels/1f/a0/9e/4ac2ef74b06f5b3d0b02def04c97c944c80d48bfbfc703db2e
Successfully built lm_eval
  Attempting uninstall: lm_eval
    Found existing installation: lm_eval 0.4.8
    Uninstalling lm_eval-0.4.8:
      Successfully uninstalled lm_eval-0.4.8
ERROR: pip's 

# Fine-Tuning


In [5]:
from unsloth import FastLanguageModel
from transformers import BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import torch
from datasets import load_dataset
import json

model_name = MODEL_UID.split("/")[-1]
dataset_name = DATASET_UID.split("/")[-1]

base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_UID, max_seq_length=2048, dtype=None, load_in_4bit=True
)

base_model.save_pretrained(f"{model_name}")
tokenizer.save_pretrained(f"{model_name}")

model = FastLanguageModel.get_peft_model(
    base_model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    # random_state = 3407,
    max_seq_length=2048,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

model.config.quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

train_dataset = load_dataset(DATASET_UID, split="train")
print(train_dataset[0].keys())
print(json.dumps(train_dataset[0]["messages"], indent=2))


def formatting_func(examples):
    messages = examples["messages"]
    texts = [
        "".join([m["content"].strip() + "\n" for m in convo]).strip()
        for convo in messages
    ]
    return {"text": texts}


train_dataset = train_dataset.map(formatting_func, batched=True)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    args=SFTConfig(
        dataset_text_field="text",
        max_seq_length=2048,
        learning_rate=2e-4,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=MAX_STEPS,
        report_to="none",  # deactivate wandb
        output_dir="outputs",
        optim="adamw_8bit",
    ),
)

trainer.train()


model.save_pretrained(f"{model_name}_finetuned_{dataset_name}")
tokenizer.save_pretrained(f"{model_name}_finetuned_{dataset_name}")

ModuleNotFoundError: No module named 'unsloth'

# Evaluation


In [ ]:
import os
import subprocess

peft_path = f"./{model_name}_finetuned_{dataset_name}"
output_path = f"results/{model_name}_eval.json"

os.environ["HF_ALLOW_CODE_EVAL"] = "1"

command = [
    "lm_eval",
    "--model",
    "hf",
    "--model_args",
    f"pretrained=./{model_name},peft={peft_path}",
    "--tasks",
    "hellaswag,gsm8k,arc_easy,truthfulqa,winogrande,humaneval",
    "--confirm_run_unsafe_code",
    "--device",
    "cuda",
    "--batch_size",
    "auto",
    "--limit",
    "1",
    "--output_path",
    output_path,
]

subprocess.run(command)